# Модель

In [1]:
from catboost import CatBoostClassifier
from data_preprocessing import get_features
import pandas as pd

In [37]:
class Model:
    def __init__(self, id_mode='with_id'):
        self.id_mode = id_mode
        assert id_mode in ['without_id', 'with_id'], KeyError()
        self.catboost_model = CatBoostClassifier().load_model(f'./catboost_{id_mode}.model')

    def get_features(self, data: pd.Series):
        return get_features(data)

    def preproccess(self, data: pd.Series, get_features):
        data = data.drop(['f2', 'f4'])
        if get_features:
             data = self.get_features(data)
        data = data.drop(['id1', 'id2', 'id3', 'text', 'label'], errors='ignore')   
        if self.id_mode == 'without_id':
            data = data.drop(['id1_pred', 'id1_in_database', 'id2_pred', 'id2_in_database',], errors='ignore')  
        return data
        
    def predict(self, data: pd.Series, 
                get_features=True, 
                threshold = 0.5):
        data = self.preproccess(data, get_features)
        # если пользователь оставляет очень много фейков или наоборот – честных отзывов
        self.catboost_model.set_probability_threshold(threshold)
        predict = self.catboost_model.predict(data, )
        return predict

In [38]:
data = "../data/wb_school_task_2.csv.gzip"
data = pd.read_csv(data, compression='gzip')
data = data.drop_duplicates(ignore_index=True)

In [39]:
model = Model(id_mode='with_id')

In [40]:
X = data.drop('label', axis=1)
Y = data['label']

In [41]:
X.head()

,id1,id2,id3,text,f1,f2,f3,f4,f5,f6,f7,f8
0,7596126584852021591,173777575861152844,18254136755182295358,"Хотела купить на замену старых,по итогу эти у...",1,1,1.0,11,13,4.272727,18,36
1,5327406586753254371,14366783259208998381,2324030737335224220,Запах по сравнению с обычным DOVE никакой. Оно...,10,6,2.7,46,4,4.434783,90,109
2,2636757786779776109,16985421652518589249,4904562693381133981,"Кабель подошёл, хорошо работает.",4,4,4.0,14,0,4.500000,4,6
3,15432976385014516263,2629180387521298831,13541353053200353836,"Восторг! Очень приятный аромат, держится долго...",6,6,5.0,374,30,4.772727,14,15
4,11933982800034911890,14531191561111600318,9121890650636487522,Визуально все хорошо. Не пробовали. Купили в п...,19,18,5.0,6,4,5.000000,26,33


In [42]:
pred = X.apply(lambda x: model.predict(x, get_features=True), axis=1)

In [43]:
pred

0       1
1       1
2       0
3       0
4       0
       ..
3117    1
3118    0
3119    0
3120    0
3121    0
Length: 3122, dtype: int64

In [44]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report

confusion_matrix(Y, pred), f1_score(Y, pred)

(array([[1818,  479],
        [ 205,  620]], dtype=int64),
 0.6444906444906444)

In [45]:
print(classification_report(Y, pred))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84      2297
           1       0.56      0.75      0.64       825

    accuracy                           0.78      3122
   macro avg       0.73      0.77      0.74      3122
weighted avg       0.81      0.78      0.79      3122

